#Bibliotecas e Drives

In [0]:
import pandas as pd
import numpy as np
import io
import requests
import folium
from folium import plugins
%matplotlib inline

In [2]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/covid19

/content/drive/My Drive/Colab Notebooks/covid19


In [3]:
!ls

 API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
 brasil_20200324.csv
 brasil_20200326.csv
 Brasil_20200327.html
 Brasil_covid19.html
 casos_100mil-2020.04.10.png
 covid19Brazil-script.ipynb
 covid19-script.ipynb
 covid19-SEIR.ipynb
'Documento sem título.gdoc'
 estimativa_brasil-2020.04.10.png
 evolução-global-2020.04.10.png
 full_data_20200320.csv
 full_data_20200321.csv
 full_data_20200322.csv
 full_data_20200323.csv
 full_data_20200326.csv
 Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
 Metadata_Indicator_API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
 novos-casos-global-2020.04.10.png
 taxa-mortes-global-2020.04.10.png
 teste.png


#Carregando o banco de dados

In [4]:
url='https://raw.githubusercontent.com/wcota/covid19br/master/cases-gps.csv'
data = requests.get(url).content
brazil_df = pd.read_csv(io.StringIO(data.decode('utf-8')))
brazil_df

,type,name,lat,lon,total
0,1,Abaetetuba/PA,-1.721828,-48.878843,2
1,1,Abaiara/CE,-7.360781,-39.048788,1
2,1,Abreu e Lima/PE,-7.900719,-34.898389,1
3,1,Acrelândia/AC,-10.075917,-67.052690,9
4,1,Adustina/BA,-10.535887,-38.114479,1
...,...,...,...,...,...
1041,1,Várzea Grande/MT,-15.645816,-56.132218,6
1042,1,Águas Lindas de Goiás/GO,-15.739228,-48.279336,1
1043,1,Águas Mornas/SC,-27.696316,-48.824783,1
1044,1,Águas de Lindóia/SP,-22.471886,-46.629103,1


In [5]:
brazil_df['mortes'] = pd.np.where(brazil_df['type'].str.contains('D1'), brazil_df['total'], 0)
brazil_df['total'] = pd.np.where(brazil_df['type'].str.contains('D1'), 0, brazil_df['total'])

brazil_df.head(15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


,type,name,lat,lon,total,mortes
0,1,Abaetetuba/PA,-1.721828,-48.878843,2,0
1,1,Abaiara/CE,-7.360781,-39.048788,1,0
2,1,Abreu e Lima/PE,-7.900719,-34.898389,1,0
3,1,Acrelândia/AC,-10.075917,-67.052690,9,0
4,1,Adustina/BA,-10.535887,-38.114479,1,0
5,D1,Adustina/BA,-10.535887,-38.114479,0,1
6,1,Afonso Cláudio/ES,-20.077841,-41.126060,1,0
7,1,Agudos/SP,-22.471162,-48.987822,1,0
8,D1,Agudos/SP,-22.471162,-48.987822,0,1
9,1,Alagoinhas/BA,-12.133526,-38.420772,5,0


In [6]:
brazil_df['total'].unique()

array([   2,    1,    9,    0,    5,    3,    4,   16,   10,   12,   24,
         32,    6,    8,   13,   11,   27,   48,   42,   23,  310,  114,
         59,   45,   15,  528,   25,    7,   31,   20,   22,   83,   14,
         34,   46,   37,   58,  205,   55,   19,   38,   44,   28,  166,
       1053,   26,   17,  102,  154,   18,  267,   82,  279,   90,   35,
         49,   40,   51,   97,  800,   63,  116,  106,   50,  107,   30,
        303,  340,   36, 1449,  314,  130,   64,  164,   41,   81,  247,
       5477,   67,   87])

## Plotando sobre o mapa


In [0]:
#Definindo o tamanho dos circulos
brazil_df['size']=brazil_df['total'].apply(lambda total:12 if total>=900 else
                                           10 if total>=400 and total<900 else 
                                           9 if total>=200 and total<400 else 3)

#plotando o mapa, em PB
mapa = folium.Map(location=[-15.788497,-47.879873],tiles='CartoDB dark_matter', 
                  zoom_start=4)

#Marcação dos pontos
for lat,lon,cidade,total,mortes,size,tipo in zip(brazil_df['lat'], brazil_df['lon'],
                                                 brazil_df['name'], brazil_df['total'],
                                                 brazil_df['mortes'],brazil_df['size'],
                                                 brazil_df['type']):
  if 'NÃO' in cidade:
    texto = "{}<br> Casos: {:,}"
    cor = 'orange'
    texto = texto.format(cidade, total)
  elif (tipo == 'D1') or (tipo == 'D0') :
    texto = "{}<br> Óbitos: {:,}"
    cor = 'black'
    texto = texto.format(cidade, mortes)
    size = 0.1
  else:
    texto = "{}<br> Casos: {:,}"
    cor = 'red'
    texto = texto.format(cidade, total)

  folium.CircleMarker([lat,lon], radius=size, popup=texto, color=cor, 
                      fill=True, fill_opacity=0.9, fill_color='darkred').add_to(mapa)

txt = "Total no Brasil<br>de casos: 17.857<br>de mortes: 941<br>data: 10/04/2020"
popup = folium.Popup(txt, max_width=300)
icon = folium.Icon(color='black', icon='info-sign')
marcador = folium.Marker(location=[-15.6914,-25.8008], popup=popup, icon=icon)

mapa.add_child(marcador)

#Mapa de calor
#mapa_calor = brazil_df[['lat', 'lon', 'total']].as_matrix()
#mapa.add_children(plugins.HeatMap(mapa_calor, radius=15))

mapa.save('Brasil_covid19.html')
#mapa